In [1]:
# ===============================
# Imports and Setup
# ===============================

import numpy as np
import pandas as pd

from linearmodels.panel import PanelOLS
import statsmodels.api as sm

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

print("Libraries imported successfully.")



Libraries imported successfully.


In [2]:
# ==============================
# Synthetic Panel Data Generation
# ==============================

# Parameters
n_products = 100
n_periods = 24

# Create panel index
products = np.arange(n_products)
periods = np.arange(n_periods)

index = pd.MultiIndex.from_product(
    [products, periods],
    names=["product_id", "time"]
)

df = pd.DataFrame(index=index).reset_index()

# Product fixed effects (unobserved heterogeneity)
product_fe = np.random.normal(0, 0.5, n_products)
df["product_fe"] = df["product_id"].map(dict(zip(products, product_fe)))

# Time fixed effects (common shocks)
time_fe = np.random.normal(0, 0.3, n_periods)
df["time_fe"] = df["time"].map(dict(zip(periods, time_fe)))

# Simulated prices
df["price"] = np.exp(
    np.random.normal(loc=2.0, scale=0.2, size=len(df))
)

# True price elasticity
true_elasticity = -1.5

# Log demand
df["log_quantity"] = (
    5
    + true_elasticity * np.log(df["price"])
    + df["product_fe"]
    + df["time_fe"]
    + np.random.normal(scale=0.3, size=len(df))
)

# Quantity in levels
df["quantity"] = np.exp(df["log_quantity"])

df.head()


,product_id,time,product_fe,time_fe,price,log_quantity,quantity
0,0,0,0.248357,-0.424611,8.309267,1.924229,6.849867
1,0,1,0.248357,-0.126194,11.451161,1.609833,5.001977
2,0,2,0.248357,-0.102814,6.061109,2.568732,13.049267
3,0,3,0.248357,-0.240683,6.597828,2.359618,10.586911
4,0,4,0.248357,-0.048386,7.537799,2.787039,16.232883


In [3]:
# ================================
# Panel Fixed-Effects Estimation
# ================================

# Set panel index
panel_df = df.set_index(["product_id", "time"])

# Log-log specification
y = panel_df["log_quantity"]
X = panel_df[["price"]]
X = sm.add_constant(np.log(X))

# Fixed-effects model with product and time effects
fe_model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True
)

fe_results = fe_model.fit(
    cov_type="clustered",
    cluster_entity=True
)

fe_results.summary


Dep. Variable:,log_quantity,R-squared:,0.4858
Estimator:,PanelOLS,R-squared (Between):,-0.0307
No. Observations:,2400,R-squared (Within):,0.3094
Date:,"Mon, Dec 15 2025",R-squared (Overall):,0.1616
Time:,22:25:33,Log-likelihood,-460.29
Cov. Estimator:,Clustered,,
,,F-statistic:,2149.9
Entities:,100,P-value,0.0000
Avg Obs:,24.000,Distribution:,"F(1,2276)"
Min Obs:,24.000,,
Max Obs:,24.000,F-statistic (robust):,2372.7


In [4]:
# Extract estimated elasticity
estimated_elasticity = fe_results.params["log(price)"]

print(f"Estimated price elasticity of demand: {estimated_elasticity:.3f}")

if estimated_elasticity < -1:
    print("Demand is elastic in the observed range.")
else:
    print("Demand is inelastic in the observed range.")


KeyError: 'log(price)'

In [ ]:
# Baseline revenue
df["revenue"] = df["price"] * df["quantity"]
baseline_revenue = df["revenue"].mean()

# Price change scenarios
price_changes = [0.01, 0.02, 0.05]
simulation_results = []

for pct in price_changes:
    new_price = df["price"] * (1 + pct)
    new_quantity = df["quantity"] * (1 + pct) ** estimated_elasticity
    new_revenue = (new_price * new_quantity).mean()

    simulation_results.append({
        "price_change_pct": f"{int(pct*100)}%",
        "avg_revenue": new_revenue,
        "pct_change_vs_baseline": (new_revenue / baseline_revenue - 1) * 100
    })

simulation_df = pd.DataFrame(simulation_results)
simulation_df


In [ ]:
simulation_df["price_change_numeric"] = simulation_df["price_change_pct"].str.replace("%","").astype(int)

plt.figure()
plt.bar(
    simulation_df["price_change_numeric"],
    simulation_df["pct_change_vs_baseline"]
)
plt.axhline(0)
plt.xlabel("Price Change (%)")
plt.ylabel("Revenue Change (%)")
plt.title("Revenue Impact of Price Increases")
plt.show()


# Pricing Elasticity Estimation with Panel Fixed Effects

This notebook estimates price elasticity of demand using panel data and fixed-effects models, and evaluates revenue implications of alternative pricing scenarios in a marketplace setting.

